## Classification

In this notebook, we will explore the concepts of clasification using the HDB data. Let's consider the following scenario:

### Scenario

The same database administrator (from the regression scenario) also mistakenly deleted the column containing data on Flat Type from 2015.  While backups exist, these data are critical to HDB’s daily operations, and time would be needed to restore these data from the backup.  HDB would like you to create a model to predict flat type given a transaction’s other characteristics.

- Assume that all data on Flat Type is missing from 2015. Ignore records beyond 2015.
- You should train your model based on data up to and including the month value of “2014-12”, then use your model to predict flat type from “2015-01” to “2015-12”.

Now, since Flat Type is a categorical variable, we will use classification to predict what the 2015 flat types are. The data does contain the ground truth which we will use to assess how close our predicted types are compared to the actual types. 

### So... what is classification?

Classification is the process of predicting the class of given data points. Classes are sometimes called as targets/ labels or categories. Classification predictive modeling is the task of approximating a mapping function ($f$) from input variables ($X$) to discrete output variables or labels ($y$). Given a set of $\displaystyle N$ training examples of the form $\displaystyle \{(x_{1},y_{1}),...,(x_{N},\;y_{N})\}$ such that $\displaystyle x_{i}$ is the feature vector of the $i$-th example and $\displaystyle y_{i}$ is its label (i.e., class), a learning algorithm seeks a function $\displaystyle f:X\to Y$, where $\displaystyle X$ is the input space and $\displaystyle Y$ is the output space. 

### Linear Classification

In linear classification, the objective is to find a linear function such that the classes are separated by lines, planes or hyperplanes. If the input feature vector to the classifier is a real vector $\vec {x}$, then the output score is
$$y=f({\vec {w}}\cdot {\vec {x}})=f\left(\sum _{j}w_{j}x_{j}\right),$$
where $\vec {w}$ is a real vector of weights and f is a function that converts the dot product of the two vectors into the desired output. The weight vector $\vec {w}$ is learned from a set of labeled training samples. Often $f$ is a threshold function, which maps all values of $\displaystyle {\vec {w}}\cdot {\vec {x}}$ above a certain threshold to the first class and some values to the second class and some values to the third class etc; e.g.,
$$\displaystyle f(\mathbf {x} )={\begin{cases}N&{\text{if }}\ \mathbf {w} \cdot \mathbf {x} >T_N,\\
N-1&{\text{if }}\ \mathbf {w} \cdot \mathbf {x} >T_{N-1},\\
\bullet\\
\bullet\\
\bullet\\
1&{\text{if }}\ \mathbf {w} \cdot \mathbf {x} >T_{1},\\
0&{\text{otherwise}}\end{cases}}$$
A more complex $f$ might give the probability that an item belongs to a certain class. 
<img src="images/Linear_Classification.jpg" alt="Drawing" style="width: 400px;"/>

#### Logistic Regression Classifier

The logistic regression model uses the logistic function to squeeze the output of a linear equation between 0 and 1. The logistic function is defined as:
$$logistic(x)= \frac {1}{1+e^{-x}}$$
And it looks like this:
<img src="images/Logistic_Function.png" alt="Drawing" style="width: 400px;"/>
The step from linear regression to logistic regression is kind of straightforward. In the linear regression model, we have modelled the relationship between outcome (for this case, assume it is categorical like 0 and 1) and features with a linear equation:
$$\hat{y} = \beta_0 + \beta_{1}x_1 + \beta_{2}x_2 + · · · + \beta_{p}x_p$$
For classification, we prefer probabilities between 0 and 1, so we wrap the right side of the equation into the logistic function. This forces the output to assume only values between 0 and 1.
$$P(Y=1|x_{1},x_{2},...,x_{p}) = \frac {e^{\beta _{0}+\beta _{1}x_{1}+\beta _{2}x_{2}+...+\beta _{p}x_{p}}}{1 + e^{\beta _{0}+\beta _{1}x_{1}+\beta _{2}x_{2}+...+\beta _{p}x_{p}}} = \frac {1}{1+e^{-(\beta _{0}+\beta _{1}x_{1}+\beta _{2}x_{2}+...+\beta _{p}x_{p})}}$$
After some tweaking with the equations and taking logarithm on both sides of the equation:
$$\log {\frac {P(Y=1|x_{1},x_{2},...,x_{p})}{1-P(Y=1|x_{1},x_{2},...,x_{p})}} = \beta _{0}+\beta _{1}x_{1}+\beta _{2}x_{2}+...+\beta _{p}x_{p}$$
The left-hand side is called the log-odds or logit. We see that the logistic regression model has a logit that is linear in X and looks very much like a typical linear regression model equation. The regression coeffients can be modeled using the least squares approach but typically, the coefficients are estimated using maximum likelihood since it has better statistical
properties. The approach to finding the coefficients using maximum likelihood is very mathematical in nature and will not be covered. For those interested, there are numerous references on the internet to read regarding this.

We can see that the value of the logistic function always lies between 0 and 1. The value is exactly 0.5 at X=0. We can use 0.5 as the probability threshold to determine the classes. If the probability is greater than 0.5, we classify it as Class-1(Y=1) or else as Class-0(Y=0).

#### So let's run logistic regression on the data to answer the question...

The first step is to import the required Python libraries.

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
DATA_DIR = './Data/'

In [ ]:
# Read CSV file and inspect columns
df = pd.read_csv(DATA_DIR+'HDB_Resale_Data_cleaned.csv',dtype= {'Postal Code': str})
df['month'] = pd.to_datetime(df['month'], format="%Y-%m-%d")
df['lease_commence_date'] = pd.to_datetime(df['lease_commence_date'], format="%Y-%m-%d")
df.columns

#### Feature Selection

Now, not all columns are needed as features. With a bit of intuition and some general knowledge, we select features that we think are good indicators of flat types. Just a recap - flat types refer to the type of HDB flats i.e. 1-room, 2-room, 3-room, 4-room, 5-room and executive. The glaring feature that distinguishes the flat types is floor area. This is intuitive as it is understood that 5-room flats have to be larger than 3-room which in turn have to be larger than 1-room. Now, flat model is another indicator of flat type. For instance, maisonettes are typically large which is indicative of 5-room or executive flats. The remaining features do not give any clear indication of flat type and therefore, we will drop them (month is kept as we need to split the data into its train and test set).

In [ ]:
df1 = df[['flat_model','flat_type','floor_area_sqm','month']]
df1.columns

In [ ]:
y = df1[['flat_type','month']]
df1 = df1.drop(['flat_type'],axis=1)
df1 = pd.get_dummies(df1)
df1.columns

Create training set from preceding 5 years i.e. 2010 - 2014.

In [ ]:
## To do
## Create training set


In [ ]:
# Solution
df_train = df1[(df1['month'].dt.year > 2009) & (df1['month'].dt.year < 2015)]
y_train = y[(y['month'].dt.year > 2009) & (y['month'].dt.year < 2015)]
df_train = df_train.drop(['month'], axis = 1)
y_train = y_train.drop(['month'],axis=1).values.reshape(-1,)
df_train.head()

Create test set from 2015 data.

In [ ]:
## To do
## Create test set


In [ ]:
# Solution
df_test = df1[(df1['month'].dt.year == 2015)]
y_test = y[(y['month'].dt.year == 2015)]
df_test = df_test.drop(['month'], axis = 1)
y_test = y_test.drop(['month'],axis=1).values.reshape(-1,)
df_test.head()

Train a Logistic Regression classifier with 3-fold cross validation. Here, we use a multi-class option as we have more than 2 classes in this problem.

In [ ]:
model = LogisticRegressionCV(multi_class='auto',solver='lbfgs',max_iter=250,verbose=1,n_jobs=-1,cv=3)
model.fit(df_train,y_train)

Generate predictions for test data.

In [ ]:
## To do
## Generate predictions on test data. Name the output as predictions


In [ ]:
# Solution
predictions = model.predict(df_test)
predictions

So, once the predictions are generated, how do we evaluate the performance of the model? In classification, there are several metrics we can use for model evaluation. Each has its pros and cons. In this workshop, we will focus on 3 metrics - precision-recall, F1-score and accuracy.

#### Precision - Recall & Accuracy
Just consider a 2-class problem.
![Precison Recall](images/Precision_Recall.png)
Note:
    - True Positive = Labelled postive data point correctly identified as positive by model
    - False Negative = Labelled postive data point incorrectly identified as negative by model
    - True Negative = Labelled negative data point correctly identified as negative by model
    - False Positve = Labelled negative data point incorrectly identified as positive by model
Precision means the percentage of results which are relevant. On the other hand, recall refers to the percentage of total relevant results correctly classified by the model. Ideally, the model should give high precision and high recall. However, in a practical situation, unless the features are perfect, this is not achievable. There is always a trade-off between precision and recall. In use cases where a missed detection is costly (like detecting heart failure), false negatives have to be minimized and therefore, the target of the model would be to go for high recall. However, the trade-off is that the precision would drop as in order to detect all the positives, the threshold will have to be lowered and more negatives will be misclassified as positives (false positives will rise). In use cases where a false detection is costly (like search results), false positives have to be minimized and therefore, the model would target for high precision. However, now the trade-off is that the recall would drop as in order to minimize misclassifications of negative samples, the threshold will have to set higher which will result in more missed detection of the positives (false negatives would rise).

Accuracy is the ratio of the number of correct predictions to all predictions by the model. Typically, accuracy is not very reflective of a model's performance as it can be misleading if the data is imbalanced. Consider 1000 passengers going through an airport. Among one of them is a terrorist. The model correctly predicts that 998 are not terrorists and misclassifies the 1 terrorist as a non-terrorist. Now, the accuracy of the model by definition is 99.8\% but this is gross misrepresentation of the model performance as its recall and precision are both 0!

#### F1-Score
In most problems, either a higher priority is given to maximizing precision, or recall, depending upon the problem being solved. There is another metric which takes into account both precision and recall, and therefore, can aim to maximize this number to make the model better. This metric is known as F1-score, which is simply the harmonic mean of precision and recall.
$$F1-Score = 2*\frac{Precision*Recall}{Precision+Recall}$$

In [ ]:
print(classification_report(y_test,predictions))
print("Accuracy:",accuracy_score(y_test, predictions))

Not bad! With a linear classifier, we can get a precision-recall of 98\%, F1-Score of 98\% and accuracy of 98.4\%. Let's see if non-linear classification can improve on these scores.

### Non-linear Classification

Predicting with logistic regression classifier returns an average precision-recall of 98\% and an accuracy of 98.5\%. Note that logisitic regression is a linear classifier. The precision, recall and accuracy rates indicate that there are still some misclassifications occurring which is indicative of an imperfect decision boundary. So there is a possibility that the decision boundary between classes is a non-linear one.
![Nonlinear Classification](images/Nonlinear_Classification.jpg)
Here, we will once again use Random Forest as the non-linear classifier. Random forest, as a classifier, uses each tree in the forest to predict the category the new record belongs to. Finally, the new record is assigned to the category that wins the majority vote.
![Random Forest Classifier](images/Random_Forest.jpg)

#### So let's run random forest classifier on the data to answer the question...

The first step is to import the required Python libraries.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
rf_eg = RandomForestClassifier(random_state = 0)

from pprint import pprint
# Look at parameters used by random forest
print('Parameters currently in use:\n')
pprint(rf_eg.get_params())

In [ ]:
base_model = RandomForestClassifier(n_estimators = 50, random_state = 0)
base_model.fit(df_train, y_train)

Run the base model on the test data.

In [ ]:
## To do
## Predict on test set. Name the output as predictions_base


In [ ]:
# Solution
predictions_base = base_model.predict(df_test)
predictions_base

In [ ]:
print(classification_report(y_test,predictions_base))
print("Accuracy:",accuracy_score(y_test, predictions_base))

In [ ]:
param_grid = { 
    'n_estimators': list(range(40,64,2))
}

# Use the grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 5 fold cross validation, 
# search across all different combinations, and use all available cores
rf_grid = GridSearchCV(estimator = rf, param_grid = param_grid, scoring='accuracy', cv = 3, verbose=2, n_jobs = -1)
# Fit the random search model
rf_grid.fit(df_train, y_train)

In [ ]:
rf_grid.best_params_

In [ ]:
best_rf = rf_grid.best_estimator_

Use the trained random forest model to predict on the test set.

In [ ]:
## To do
## Predict on test set. Name the output as predictions


In [ ]:
# Solution
predictions = best_rf.predict(df_test)
predictions

In [ ]:
print(classification_report(y_test,predictions))
print("Accuracy:",accuracy_score(y_test, predictions))

With Random Forest, we get a precision-recall of 99\%, F1-Score of 99\% and accuracy of 99.2\%. Marginal improvement over the linear classifier at the cost of increased computation power and time. 

Using a non-linear classifer does not always guarantee superior performance. If a high enough precision-recall can already be achieved by a linear classifier, the model is sufficient for the use-case.